In [ ]:
from vigilant.extensions import InfluxClient
client = InfluxClient('', 'telegraf')

In [ ]:
client.query('SHOW MEASUREMENTS ON telegraf')

In [ ]:
res = client.query('SELECT * FROM telegraf..test')
res['results']

In [ ]:
client.post('DROP DATABASE telegraf')

In [ ]:
import textwrap, requests, json
def compare(measurement, field, database, rp='autogen'):
    # create and run a task comparing the passed field against its max value
    task_id = field.replace('/', '-')
    script = textwrap.dedent(f"""
        stream
            |from()
                .measurement('test')
            |log()
            |alert()
                .crit(lambda: "{field}" >  "{field}:max")
                .message('out of range')
                .details('')
                .log('/tmp/alerts.txt')
    """)
    task = {
        "id" : task_id,
        "type" : "stream",
        "dbrps": [{"db": database, "rp" : rp}],
        "script": script,
        "status": "enabled"
    }
    requests.delete(f'http://localhost:9092/kapacitor/v1/tasks/{task_id}')

    r = requests.post('http://localhost:9092/kapacitor/v1/tasks', data=json.dumps(task))
    return r

compare('test', 'default/read_voltage', 'telegraf').text
    
    
    


In [ ]:
json.dumps(task)

In [ ]:
import os
os.environ['KAPACITOR_DATA_DIR'] = "/Users/robbiefasano/.kapacitor"